In [5]:
import numpy as np
from collections import defaultdict

class GridWorld:
    def __init__(self, size=5):
        self.size = size
        self.state = 0
        self.goal_state = size * size - 1
        
    def reset(self):
        self.state = 0
        return self.state
    
    def step(self, action):
        # Actions: 0: right, 1: down, 2: left, 3: up
        x, y = self.state % self.size, self.state // self.size
        
        if action == 0:   # Right
            x = min(x + 1, self.size - 1)
        elif action == 1: # Down
            y = min(y + 1, self.size - 1)
        elif action == 2: # Left
            x = max(x - 1, 0)
        elif action == 3: # Up
            y = max(y - 1, 0)
            
        new_state = y * self.size + x
        reward = 1.0 if new_state == self.goal_state else 0.0
        done = new_state == self.goal_state
        
        self.state = new_state
        return new_state, reward, done

def extract_features(state, action, size):
    # One-hot encoding of state-action pairs
    features = np.zeros(size * size * 4)
    features[state * 4 + action] = 1
    return features

def least_squares_value_iteration(env, gamma=0.99, num_episodes=1000):
    size = env.size
    num_actions = 4
    feature_size = size * size * num_actions
    
    theta = np.zeros(feature_size)
    A = np.zeros((feature_size, feature_size))
    b = np.zeros(feature_size)
    
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        
        while not done:
            # Choose action (epsilon-greedy)
            if np.random.random() < 0.1:  # exploration
                action = np.random.randint(num_actions)
            else:  # exploitation
                q_values = [np.dot(theta, extract_features(state, a, size)) for a in range(num_actions)]
                action = np.argmax(q_values)
            
            # Take action
            next_state, reward, done = env.step(action)
            
            # Extract features
            features = extract_features(state, action, size)
            
            if done:
                target = reward
            else:
                next_q_values = [np.dot(theta, extract_features(next_state, a, size)) for a in range(num_actions)]
                target = reward + gamma * max(next_q_values)
            
            # Update A and b matrices
            A += np.outer(features, features)
            b += target * features
            
            # Update state
            state = next_state
        
        # Solve for theta using least squares
        if episode % 10 == 0:  # Update theta periodically
            theta = np.linalg.solve(A + 0.1 * np.eye(feature_size), b)
    
    return theta

def test_policy(env, theta):
    state = env.reset()
    total_reward = 0
    done = False
    
    while not done:
        q_values = [np.dot(theta, extract_features(state, a, env.size)) for a in range(4)]
        action = np.argmax(q_values)
        state, reward, done = env.step(action)
        total_reward += reward
    
    return total_reward


In [6]:
# Create environment and run algorithm
env = GridWorld(size=5)
theta = least_squares_value_iteration(env)

# Test the learned policy
total_reward = test_policy(env, theta)
print(f"Total reward with learned policy: {total_reward}")

# Print value function
print("\nValue function:")
for y in range(env.size):
    row = ""
    for x in range(env.size):
        state = y * env.size + x
        value = max([np.dot(theta, extract_features(state, a, env.size)) for a in range(4)])
        row += f"{value:.2f}\t"
    print(row)

Total reward with learned policy: 1.0

Value function:
0.36	0.56	0.65	0.34	0.27	
0.54	0.73	0.86	0.80	0.72	
0.62	0.78	0.94	0.97	0.98	
0.00	0.64	0.84	0.99	1.00	
0.00	0.00	0.89	0.99	0.00	
